In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, datasets


from PIL import Image 
from torch.utils.data import DataLoader
import os 

In [2]:
path = "./Data/aug_data/training/training"
val_path = "./Data/aug_data/validation/validation"

In [3]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [4]:
train_data = datasets.ImageFolder(root=path, transform=transform)
test_data = datasets.ImageFolder(root=val_path, transform=transform)

In [5]:
# train_data.target_transform = lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)

In [6]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [7]:
torch.cuda.empty_cache()

In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)  #3, 224, 224  ----> 32, 220, 220
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2,2)    #32, 220, 220  ----> 32, 110, 110
        self.conv2 = nn.Conv2d(32, 64, 5) #32, 110, 110, ----> 64, 106, 106  ----> 64, 53, 53
        self.bn2 = nn.BatchNorm2d(64)
        self.f1    = nn.Linear(64*53*53, 120)
        self.dropout = nn.Dropout(0.5)
        self.f2    = nn.Linear(120, 84)
        self.f3    = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.bn1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.bn2(x)
        x = x.view(-1, 64*53*53)
        x = F.relu(self.f1(x))
        x = self.dropout(x)
        x = F.relu(self.f2(x))
        x = self.f3(x)
        return x



In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
lr = 0.001
epochs = 5

total_steps = len(train_loader)
model = ConvNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [14]:
for epoch in range(epochs):
    
    running_loss = 0.
    
    n_correct_test = 0
    n_samples_test = 0
    model.train()
    for i , (images, labels ) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        pred = model(images)
        
        loss = loss_fn(pred, labels) 

        running_loss += loss.item()
        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")
    

    model.eval()
    with torch.no_grad():
        
        for i, (imagess, labelss) in enumerate(test_loader):
            imagess = imagess.to(device)
            labelss = labelss.to(device)

            out = model(imagess)
            _, predicted = torch.max(out, 1)
            n_samples_test += labelss.size(0)
            n_correct_test += (predicted ==labelss).sum().item()



    acc_test = 100.0 * n_correct_test / n_samples_test
    
        



    print(f"test_accuracy:{acc_test}")

print("Finished Training ")
        
        

Epoch 1, Loss: 1.4184043751313136
test_accuracy:50.36764705882353
Epoch 2, Loss: 1.2964081729833896
test_accuracy:51.470588235294116
Epoch 3, Loss: 1.2076863646507263
test_accuracy:65.07352941176471
Epoch 4, Loss: 1.0277735728483934
test_accuracy:61.029411764705884
Epoch 5, Loss: 0.964575469493866
test_accuracy:56.98529411764706
Finished Training 


In [15]:
model.eval()
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)

        out = model(images)

        _, predicted = torch.max(out, 1)
        n_samples += labels.size(0)
        n_correct += (predicted ==labels).sum().item()

        for j in range(len(labels)):
            label = labels[j]
            pred = predicted[j]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    acc = 100.0 * n_correct / n_samples
    print(f"Total Accuracy : {acc}")

    for i in range(10):
        acc1 = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f"Class[{i}] Accuracy : {acc1}")
            

Total Accuracy : 56.98529411764706
Class[0] Accuracy : 38.46153846153846
Class[1] Accuracy : 75.0
Class[2] Accuracy : 92.5925925925926
Class[3] Accuracy : 56.666666666666664
Class[4] Accuracy : 46.15384615384615
Class[5] Accuracy : 39.285714285714285
Class[6] Accuracy : 34.61538461538461
Class[7] Accuracy : 17.857142857142858
Class[8] Accuracy : 88.88888888888889
Class[9] Accuracy : 80.76923076923077
